In [2]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os


def load_labeled_data(filename):
    """Load labeled data from a JSON file."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
            labeled_data = [(item.get("data"), item.get("label")) for item in data]
            return pd.DataFrame(labeled_data, columns=["data", "labels"])
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
    except json.JSONDecodeError as e:
        print(f"Error: Unable to decode JSON file '{filename}': {e}")
    except Exception as e:
        print(f"Error: An unexpected error occurred: {e}")

    return None


def train_model(X_train, y_train, vectorizer=None):
    """Train or update a Naive Bayes model."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)  # Fit the vectorizer with training data
    X_train_vectorized = vectorizer.transform(X_train)

    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)

    return model, vectorizer


def save_custom_dictionary(filename, vocabulary):
    """Save custom dictionary as JSON."""
    with open(filename, "w") as file:
        json.dump(list(set(vocabulary)), file)


def check_spelling(spell, words_to_check):
    """Check spelling and get suggestions."""
    corrected_words = [spell.correction(word) for word in words_to_check]
    suggestions = [spell.candidates(word) for word in words_to_check]
    return corrected_words, suggestions


def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    # Initialize the spell checker and load the custom dictionary
    spell = SpellChecker(language="en")
    if os.path.exists(custom_dictionary_filename):
        spell.word_frequency.load_dictionary(custom_dictionary_filename)

    # Initialize an empty list to keep track of misclassified samples
    misclassified_samples = []
    misclassification_threshold = 5
    epochs = 0

    # Initialize empty DataFrames to store all training data
    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    while epochs < 20:
        labeled_df = load_labeled_data(labeled_data_filename)
        if labeled_df is None:
            return

        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        all_X_train = pd.concat([all_X_train, X_train], ignore_index=True)
        all_y_train = pd.concat([all_y_train, y_train], ignore_index=True)

        model, vectorizer = train_model(all_X_train, all_y_train)

        # Check for misclassified samples
        words_to_check = ["motday", "hgost", "mney"]
        predictions = model.predict(vectorizer.transform(words_to_check))
        misclassified_samples.extend(
            [
                word
                for word, prediction in zip(words_to_check, predictions)
                if not prediction
            ]
        )

        if len(misclassified_samples) >= misclassification_threshold:
            break

        epochs += 1

    # Save custom dictionary incrementally
    new_vocabulary = vectorizer.get_feature_names_out()
    if os.path.exists(custom_dictionary_filename):
        with open(custom_dictionary_filename, "r") as file:
            existing_vocabulary = set(json.load(file))
        new_vocabulary = existing_vocabulary.union(new_vocabulary)
    with open(custom_dictionary_filename, "w") as file:
        json.dump(list(new_vocabulary), file)

    # Save misclassified samples
    with open(misclassified_samples_filename, "w") as file:
        json.dump(misclassified_samples, file)

    # Check spelling and print corrected words and suggestions
    corrected_words, suggestions = check_spelling(spell, words_to_check)
    for word, suggestion_list in zip(corrected_words, suggestions):
        print(f"Original: {word}, Corrected: {word}, Suggestions: {suggestion_list}")

    print("Training loop finished.")


if __name__ == "__main__":
    main()

Original: today, Corrected: today, Suggestions: {'moldy', 'moody', 'modal', 'midday', 'today', 'mothy', 'moray', 'mayday', 'motley'}
Original: ghost, Corrected: ghost, Suggestions: {'host', 'ghost'}
Original: money, Corrected: money, Suggestions: {'money'}
Training loop finished.


In [12]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os


def load_labeled_data(filename):
    """Load labeled data from a JSON file."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
            labeled_data = [(item.get("data"), item.get("label")) for item in data]
            return pd.DataFrame(labeled_data, columns=["data", "labels"])
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
    except json.JSONDecodeError as e:
        print(f"Error: Unable to decode JSON file '{filename}': {e}")
    except Exception as e:
        print(f"Error: An unexpected error occurred: {e}")

    return None


def save_custom_dictionary(filename, new_vocabulary):
    """Save custom dictionary as JSON."""
    custom_dictionary = set()

    # Load existing custom dictionary if it exists
    if os.path.exists(filename):
        with open(filename, "r") as file:
            existing_vocabulary = set(json.load(file))
        # Union existing vocabulary with new vocabulary
        custom_dictionary.update(existing_vocabulary)

    # Update custom dictionary with new vocabulary
    custom_dictionary.update(new_vocabulary)

    # Save the updated custom dictionary
    with open(filename, "w") as file:
        json.dump(list(custom_dictionary), file)


def check_spelling(spell, words_to_check):
    """Check spelling and get suggestions."""
    # Convert the list of words to a pandas Series
    words_series = pd.Series(words_to_check)

    # Use apply method to apply the correction function to each word
    corrected_words_series = words_series.apply(spell.correction)

    # Use apply method to apply the candidates function to each word
    suggestions_series = words_series.apply(spell.candidates)

    # Convert the corrected words and suggestions back to lists
    corrected_words = corrected_words_series.tolist()
    suggestions = suggestions_series.tolist()

    return corrected_words, suggestions


def fit_vectorizer(X_train, vectorizer=None):
    """Fit or update a CountVectorizer."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)
    return vectorizer


def train_model(X_train_vectorized, y_train):
    """Train a Naive Bayes model."""
    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)
    return model


def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    # Initialize the spell checker and load the custom dictionary
    spell = SpellChecker(language="en")
    if os.path.exists(custom_dictionary_filename):
        spell.word_frequency.load_dictionary(custom_dictionary_filename)

    # Initialize an empty list to keep track of misclassified samples
    misclassified_samples = []
    misclassification_threshold = 5
    epochs = 0

    # Initialize empty DataFrames to store all training data
    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    while epochs < 20:
        labeled_df = load_labeled_data(labeled_data_filename)
        if labeled_df is None:
            return

        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        all_X_train = pd.concat([all_X_train, X_train], ignore_index=True)
        all_y_train = pd.concat([all_y_train, y_train], ignore_index=True)

        vectorizer = fit_vectorizer(all_X_train)

        X_train_vectorized = vectorizer.transform(all_X_train)

        model = train_model(X_train_vectorized, all_y_train)

        # Check for misclassified samples
        words_to_check = ["motday", "hgost", "mney"]
        predictions = model.predict(vectorizer.transform(words_to_check))
        misclassified_samples.extend(
            [
                word
                for word, prediction in zip(words_to_check, predictions)
                if not prediction
            ]
        )

        if len(misclassified_samples) >= misclassification_threshold:
            break

        epochs += 1

    # Save custom dictionary incrementally
    new_vocabulary = vectorizer.get_feature_names_out()
    if os.path.exists(custom_dictionary_filename):
        with open(custom_dictionary_filename, "r") as file:
            existing_vocabulary = set(json.load(file))
        new_vocabulary = existing_vocabulary.union(new_vocabulary)
    with open(custom_dictionary_filename, "w") as file:
        json.dump(list(new_vocabulary), file)

    # Save misclassified samples
    with open(misclassified_samples_filename, "w") as file:
        json.dump(misclassified_samples, file)

    # Check spelling and print corrected words and suggestions
    corrected_words, suggestions = check_spelling(spell, words_to_check)
    for word, suggestion_list in zip(corrected_words, suggestions):
        print(f"Original: {word}, Corrected: {word}, Suggestions: {suggestion_list}")

    print("Training loop finished.")


if __name__ == "__main__":
    main()

Original: today, Corrected: today, Suggestions: {'moldy', 'moody', 'modal', 'midday', 'today', 'mothy', 'moray', 'mayday', 'motley'}
Original: ghost, Corrected: ghost, Suggestions: {'host', 'ghost'}
Original: money, Corrected: money, Suggestions: {'money'}
Training loop finished.
